## Import and data load

In [1]:
import pandas as pd
import numpy as np

from bokeh.plotting import (figure,
                            output_file,
                            output_notebook,
                            show)
from bokeh.models import (ColumnDataSource, LabelSet)
from bokeh.transform import dodge

### Input data
Put the latest or preferred varicarta export in this notebook's directory.

In [2]:
variant_data = pd.read_csv("export_latest.tsv", sep="\t")

/home/y/anaconda3/envs/varicarta/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3062: DtypeWarning: Columns (22,26) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


### Prepare data for bokeh format

Should have xname, yname, colors, alphas, count.

In [3]:
data = variant_data[["paper_key", "event_id"]]
data.set_index("paper_key", inplace=True)
data = data.reset_index()
data["event_id"] = pd.Categorical(data["event_id"])
data["paper_key"] = pd.Categorical(data["paper_key"])
data = data.drop_duplicates()

### Compute overlaps

In [4]:
papers = data["paper_key"].drop_duplicates()
paper_overlap_list = [data.groupby("paper_key")["event_id"].aggregate(lambda x : x.isin(data.loc[data["paper_key"] == k,"event_id"]).sum()) for k in papers ]

#### Merge into a single dataframe

In [5]:
overlaps_df = pd.concat(paper_overlap_list, axis=1)
overlaps_df.columns = papers
overlaps_df = overlaps_df.loc[papers,papers]    

#### Unstack and make a 2-column molten dataframe.

In [6]:
overlap_unstack = pd.DataFrame( overlaps_df.stack() )
overlap_unstack.index.names= ["xname", "yname"]
overlap_unstack = overlap_unstack.reset_index()
overlap_unstack = overlap_unstack[overlap_unstack[0] > 0]

In [7]:
OVERLAP_COLOR = "dodgerblue"
DIAG_COLOR="#FFD699"
NOTHING_COLOR="white"

xname = overlap_unstack["xname"].tolist()
yname = overlap_unstack["yname"].tolist()
counts = overlap_unstack[0].tolist()

""" Set colors """
color = [OVERLAP_COLOR if x > 0 else NOTHING_COLOR for x in counts] # Active or empty
color = [ c if str(x) != str(y) else DIAG_COLOR for x,y,c in zip(xname, yname, color) ] # Paint diagonal.

""" Set alpha of color """
alpha = [ min(x,1.0) for x in np.log(overlap_unstack[0]) / 10 ]
alpha = [ a if str(x) != str(y) else 1.0 for x,y,a in zip(xname, yname, alpha) ] # Set diagonal to 1.0 alpha

""" Get a sorted list of names """
uname = sorted( list(set(xname)) )

""" Create a Column Data Source """
data=ColumnDataSource(dict(
    xname=xname,
    yname=yname,
    colors=color,
    alphas=alpha,
    count=counts,
    txtcount=[str(c) if len(str(c)) < 4 else str(c)[:-3] + "K" for c in counts]
))

""" Make figure """
p = figure(title="Variant Event Overlap Between Papers",
           x_axis_location="above", 
           tools="hover,save,box_zoom,reset",
           x_range=list(reversed(uname)), 
           y_range=uname,
           tooltips = """
           <div style="">
            <p  style='margin:0'> <strong>@xname</strong> reports  </p>
            <p  style='margin:0'> <strong>@count</strong> variant events also reported in </p>
            <p  style='margin:0'> <strong>@yname</strong>. </p>
           </div>
            """
          )
p.title.text_font_size = '18pt' ## Title size

""" Label for inside the cells of the heatmap """
labels = LabelSet(x="xname", 
                  y=dodge('yname', -0.25, range=p.y_range),
                  text='txtcount',
                  source=data,
                  text_align='center'
                 )
labels.text_font_size="10px"
labels.text_font_style = "bold"

p.plot_width = 2000
p.plot_height = 2000
p.grid.grid_line_dash_offset = 7
p.grid.grid_line_color = None
p.axis.axis_line_color = None
# p.axis.major_tick_line_color = None # Enables lines though centered on 0.0 of the factor value; needs offset to make a heatmap grid.
p.axis.major_label_text_font_size = "12px"
p.axis.major_label_standoff = 10
p.xaxis.major_label_orientation = np.pi/3

""" Add heatmap tiles """
p.rect('xname',
       'yname',
       0.9, 0.9,
       source=data,
       color='colors',
       alpha='alphas',
       dilate=True,
       line_color='black',
       hover_line_color='black',
       hover_color='colors')

""" Render labels """
p.add_layout(labels)
p.toolbar.logo = None # Remove Bokeh logo

output_file("variant_heatmap_latest.html") # Save to file.
output_notebook() # For showing below
show(p) # Show the plot

Loading BokehJS ...

#### Fix a bug with the meta tag not having a closing slash

In [8]:
%%bash
sed -i 's|\(.*meta.*\)>|\1/>|g' variant_heatmap_latest.html

#### Transfer file to static resources

In [9]:
cp variant_heatmap_latest.html ../../src/main/webapp/resources/static/variant_heatmap_latest.html